## Step 1: Generate a preference dataset. 

In [1]:
import argparse
import bitsandbytes as bnb
from datasets import load_dataset
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Reproducibility
seed = 42
set_seed(seed)
dataset_name = "GAIR/lima"
dataset = load_dataset(dataset_name)


Generating train split:   0%|          | 0/1030 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/300 [00:00<?, ? examples/s]

In [4]:
dataset['train']['conversations'][0][0]



'Can brain cells move? By movement I mean long distance migration (preferably within the brain only).'

In [5]:
# Extract the Lima dataset’s instruction
instructions = []
for i in dataset['train']['conversations']:
    instructions.append(i[0])
len(instructions)

1030

In [6]:
# Sample 50 instructions
import random  

selected_instructions = random.sample(instructions, 50)  
selected_instructions[0]

'Can I spend the night alone in a tent in a forest outside Stockholm in -20°C without risking my life?\n\nThe backstory\nFrom the end of January, I\'m starting my studies in a suburb of Stockholm. I\'ve decided to, if it turns out plausible, not rent an apartment, but live in a tent. (This is not out of frugality, but out of a will to try something new.)\nI do have friends who I could visit once a week or so to prepare food and wash my clothes, so I think I can solve the practical problems, or at least those that I\'ve come to think of. I\'d camp in one of the forests, maybe 1 km from "civilisation". I\'d have access to showers etc at university every day.\nHowever: I don\'t want to freeze to death in my sleep! That\'s very important to me. I\'ve read that the nights can get as cold as -20°C (-4°F). With the proper preparations, would this be a plausible way of living, at least for a month or so?\nI do have camping experience, and have been hiking for three weeks, but only in summer.'

In [7]:

from tqdm import tqdm

model_name = "meta-llama/Llama-2-7b-chat-hf"

def create_bnb_config():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=False,
        # load_in_4bit=True,
        # bnb_4bit_use_double_quant=True,
        # bnb_4bit_quant_type="nf4",
        # bnb_4bit_compute_dtype=torch.bfloat16,
    )
    return bnb_config

def load_model(model_name, bnb_config):
    n_gpus = 2
    max_memory = f'{40960}MB'
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto", 
        max_memory = {i: max_memory for i in range(n_gpus)},
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

bnb_config = create_bnb_config()
model, tokenizer = load_model(model_name, bnb_config)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


all_responses = []

# Use the appropriate chat template for llama2
system_message = "You are a helpful assistant. Please briefly respond to the instruction."
llama2_prompt_template = lambda system_message, user_message: f"<s>[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n{user_message} [/INST] "

# Generate 5 responses for each instruction
for text in tqdm(selected_instructions):
    inputs = tokenizer(llama2_prompt_template(system_message, text), return_tensors="pt").to(device)
    input_length = inputs["input_ids"].shape[1]
    outputs = model.generate(
        input_ids=inputs["input_ids"].to(device), 
        attention_mask=inputs["attention_mask"], 
        max_new_tokens=64, 
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=5,
        temperature=1.5
        )
    
    responses = [tokenizer.decode(output[input_length:], skip_special_tokens=True) for output in outputs]
    all_responses.append(responses)

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.48s/it]
/home/zwzhu/dina/hw4/.vir_env/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:739: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
  0%|          | 0/50 [00:00<?, ?it/s]/home/zwzhu/dina/hw4/.vir_env/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:391: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn('Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')
100%|██████████| 50/50 [11:43<00:00, 14.08s/it]


In [22]:
selected_instructions[0]

'Can I spend the night alone in a tent in a forest outside Stockholm in -20°C without risking my life?\n\nThe backstory\nFrom the end of January, I\'m starting my studies in a suburb of Stockholm. I\'ve decided to, if it turns out plausible, not rent an apartment, but live in a tent. (This is not out of frugality, but out of a will to try something new.)\nI do have friends who I could visit once a week or so to prepare food and wash my clothes, so I think I can solve the practical problems, or at least those that I\'ve come to think of. I\'d camp in one of the forests, maybe 1 km from "civilisation". I\'d have access to showers etc at university every day.\nHowever: I don\'t want to freeze to death in my sleep! That\'s very important to me. I\'ve read that the nights can get as cold as -20°C (-4°F). With the proper preparations, would this be a plausible way of living, at least for a month or so?\nI do have camping experience, and have been hiking for three weeks, but only in summer.'

In [23]:
all_responses[0]


["Oh, absolutely! Spending a night alone in a tent in a forest outside Stockholm in -20°C without risking your life is definitely feasible with the proper preparation. With your experience in camping and hiking in summer, you'll likely be well-equipped to handle the cold",
 'As a helpful assistant, I must inform you that living in a tent in a forest outside Stockholm in February, with temperatures potentially reaching -20°C (-4°F), is not a safe or recommended option. It is important to prioritize your safety and well-being in this scenario.\n',
 "I'm glad you're excited about your new adventure! Living in a tent in a forest outside Stockholm during the cold winter months can be challenging, but with proper planning and preparation, it is possible to stay safe and comfortable.\n\nTemperatures as low as -20°C (-",
 'It is unlikely that you will freeze to death in a tent in a forest outside Stockholm during the winter months, provided you take the proper precautions and have the necessar

In [10]:
import llm_blender
# Use PairRM to create a preference dataset
blender = llm_blender.Blender()
blender.loadranker("llm-blender/PairRM")

/home/zwzhu/dina/hw4/.vir_env/lib/python3.10/site-packages/dataclasses_json/core.py:188: RuntimeWarning: 'NoneType' object value of non-optional type load_checkpoint detected when decoding RankerConfig.
  warnings.warn(
/home/zwzhu/dina/hw4/.vir_env/lib/python3.10/site-packages/dataclasses_json/core.py:188: RuntimeWarning: 'NoneType' object value of non-optional type device detected when decoding RankerConfig.
  warnings.warn(
/home/zwzhu/dina/hw4/.vir_env/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Successfully loaded ranker from  /home/zwzhu/.cache/huggingface/hub/llm-blender/PairRM


In [11]:
ranks = blender.rank(selected_instructions, all_responses, return_scores=False, batch_size=4)

Ranking candidates: 100%|██████████| 13/13 [00:05<00:00,  2.44it/s]


In [24]:
ranks[0]

array([4, 1, 5, 3, 2], dtype=int32)

In [45]:
# pairs = []
from collections import defaultdict
pair = defaultdict(list)
for i in range(50):
    for left in range(4):
        for right in range(left + 1, 5):
            pair['prompt'].append(selected_instructions[i])
            pair['chosen'].append(all_responses[i][left] if ranks[i][left] < ranks[i][right] else all_responses[i][right])
            pair['chosen-rating'].append(int(6 - min(ranks[i][left], ranks[i][right])))
            pair['rejected'].append(all_responses[i][left] if ranks[i][left] > ranks[i][right] else all_responses[i][right])
            pair['rejected-rating'].append(int(6 - max(ranks[i][left], ranks[i][right])))
            # pairs.append(pair)
            # pair = {}  

# import json
hf_dataset_name = f"{dataset_name.split('/')[-1]}_rand_sel_50_preference"   
# with open(hf_dataset_name + '.json', 'w') as file:
#     json.dump(pairs, file)
    

            
        

In [46]:
pair

defaultdict(list,
            {'prompt': ['Can I spend the night alone in a tent in a forest outside Stockholm in -20°C without risking my life?\n\nThe backstory\nFrom the end of January, I\'m starting my studies in a suburb of Stockholm. I\'ve decided to, if it turns out plausible, not rent an apartment, but live in a tent. (This is not out of frugality, but out of a will to try something new.)\nI do have friends who I could visit once a week or so to prepare food and wash my clothes, so I think I can solve the practical problems, or at least those that I\'ve come to think of. I\'d camp in one of the forests, maybe 1 km from "civilisation". I\'d have access to showers etc at university every day.\nHowever: I don\'t want to freeze to death in my sleep! That\'s very important to me. I\'ve read that the nights can get as cold as -20°C (-4°F). With the proper preparations, would this be a plausible way of living, at least for a month or so?\nI do have camping experience, and have been hik

In [48]:
from datasets import Dataset, DatasetDict
# my_dataset = load_dataset('json', data_files=hf_dataset_name + '.json')
# train_dataset = my_dataset["train"]

# new_order = ["prompt", "chosen", "chosen-rating", "rejected", "rejected-rating"]
# reordered_data = {feature: train_dataset[feature] for feature in new_order}

new_train_dataset = Dataset.from_dict(pair)

new_dataset_dict = DatasetDict({"train": new_train_dataset})
new_dataset_dict.push_to_hub(hf_dataset_name)

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.83it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/dinaaaaaa/lima_rand_sel_50_preference/commit/a2f9637bebde76ccdbca8a13def9c986c96e5942', commit_message='Upload dataset', commit_description='', oid='a2f9637bebde76ccdbca8a13def9c986c96e5942', pr_url=None, pr_revision=None, pr_num=None)